## Quantum Neural Network on $3 \times 3$ downsampled MNIST Digits  
Sebastian Molina  
smolinad@unal.edu.co

In [1]:
from functools import partial
import numpy as np
import tensorflow as tf

import tensorcircuit as tc
from tensorcircuit import keras

from mnist_preprocessing import preprocess_mnist_digits as mnist

from classical_neural_network import NeuralNetwork

In [2]:
tc.set_backend("tensorflow")
tc.set_dtype("complex128")

img_height = 3
n = img_height**2 #Number of qubits and pixels in the downsampled image
nlayers = 3 
nsamples = 4000

### MNIST data Preprocessing

In [3]:
# x_train, y_train, x_test, y_test = mnist(img_height=n)
# x_train = np.squeeze(x_train)[:nsamples]
# x_train = tf.reshape(tf.constant(x_train, dtype=tf.float64), [-1, n])
# y_train = tf.constant(y_train[:nsamples], dtype=tf.float64)

# x_test = tf.reshape(tf.constant(x_test, dtype=tf.float64), [-1, n])
# y_test = tf.constant(y_test, dtype=tf.float64)

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train[..., np.newaxis] / 255.0
x_test = x_test[..., np.newaxis] / 255.0


def filter_pair(x, y, a, b):
    keep = (y == a) | (y == b)
    x, y = x[keep], y[keep]
    y = y == a
    return x, y

x_train, y_train = filter_pair(x_train, y_train, 3, 6)
x_test, y_test = filter_pair(x_test, y_test, 3, 6)

x_train_small = tf.image.resize(x_train, (img_height, img_height)).numpy()
x_train_bin = np.array(x_train_small > 0.5, dtype=np.float32)
x_train_bin = np.squeeze(x_train_bin)[:nsamples]
x_train = tf.reshape(tf.constant(x_train_bin, dtype=tf.float64), [-1, n])
y_train = tf.constant(y_train[:nsamples], dtype=tf.float64)

x_test_small = tf.image.resize(x_test, (img_height, img_height)).numpy()
x_test_bin = np.array(x_test_small > 0.5, dtype=np.float32)
x_test = tf.reshape(tf.constant(x_test_bin, dtype=tf.float64), [-1, n])
y_test = tf.constant(y_test, dtype=tf.float64)

### Quantum variational classifier (Farhi & Neven) 
No vmap or vvag optimization. Using vmap runs way faster.

In [5]:
def variational_classifier(x, weights, nlayers):

    c = tc.Circuit(n+1)

    for i in range(n):
        c.rx(i, theta=x[i])
            
    c.X(n)
    c.H(n)

    for i in range(nlayers):
        for j in range(n):
            c.exp1(j, n, unitary=tc.gates._zx_matrix, theta=weights[2*i, j])
            c.exp1(j, n, unitary=tc.gates._xx_matrix, theta=weights[2*i+1, j])
                
    c.H(n)

    ## Changing to original measurement (returning c.Z(n)) ends in bad accuracy.
    ypred = c.expectation([tc.gates.z(), (4, )]) #Why (4,)
    ypred = (tc.backend.real(ypred) + 1) / 2.0 #Why divide in two. Related: https://tensorcircuit.readthedocs.io/en/latest/tutorials/mnist_qml.html?highlight=MNIST
    return ypred


layer = keras.QuantumLayer(partial(variational_classifier, nlayers=nlayers), [(2 * nlayers, n)])

In [6]:
# Keras interface with Keras training paradigm

model = tf.keras.Sequential([layer])

def hinge_accuracy(y_true, y_pred):
    y_true = tf.squeeze(y_true) > 0.0
    y_pred = tf.squeeze(y_pred) > 0.0
    result = tf.cast(y_true == y_pred, tf.float32)

    return tf.reduce_mean(result)


model.compile(
    loss="binary_crossentropy", #Because labels are in [0, 1] range. Can be changed to hinge loss if labels are transformed to [-1, 1]. Related: https://www.tensorflow.org/quantum/tutorials/mnist#2_quantum_neural_network
    optimizer=tf.keras.optimizers.legacy.Adam(0.01),
    metrics=["binary_accuracy"],
)

model.fit(x_train, y_train, batch_size=32, epochs=20)
#[:int(nsamples/10)]

Epoch 1/20
125/125 [==============================] - 24s 46ms/step - loss: 0.6944 - binary_accuracy: 0.6398
Epoch 2/20
125/125 [==============================] - 6s 48ms/step - loss: 0.6164 - binary_accuracy: 0.6955
Epoch 3/20
125/125 [==============================] - 6s 48ms/step - loss: 0.6175 - binary_accuracy: 0.6955
Epoch 4/20
125/125 [==============================] - 7s 55ms/step - loss: 0.6173 - binary_accuracy: 0.6955
Epoch 5/20
125/125 [==============================] - 7s 53ms/step - loss: 0.6157 - binary_accuracy: 0.6955
Epoch 6/20
125/125 [==============================] - 7s 54ms/step - loss: 0.6190 - binary_accuracy: 0.6955
Epoch 7/20
125/125 [==============================] - 6s 49ms/step - loss: 0.6170 - binary_accuracy: 0.6955
Epoch 8/20
125/125 [==============================] - 6s 49ms/step - loss: 0.6165 - binary_accuracy: 0.6955
Epoch 9/20
125/125 [==============================] - 6s 49ms/step - loss: 0.6198 - binary_accuracy: 0.6955
Epoch 10/20
125/125 [======

In [7]:
model.evaluate(x_test, y_test)

62/62 [==============================] - 11s 23ms/step - loss: 0.6188 - binary_accuracy: 0.6956


[0.6188288331031799, 0.6955645084381104]

### Original example (TensorCircuit)

In [8]:
def qml_y(x, weights, nlayers):
    weights = tc.backend.cast(weights, "complex128")
    x = tc.backend.cast(x, "complex128")
    c = tc.Circuit(n)
    for i in range(n):
        c.rx(i, theta=x[i])
    for j in range(nlayers):
        for i in range(n - 1):
            c.cnot(i, i + 1)
        for i in range(n):
            c.rx(i, theta=weights[2 * j, i])
            c.ry(i, theta=weights[2 * j + 1, i])
    ypred = c.expectation([tc.gates.z(), (4,)])
    ypred = tc.backend.real(ypred)
    ypred = (tc.backend.real(ypred) + 1) / 2.0
    return ypred

ql = keras.QuantumLayer(partial(qml_y, nlayers=nlayers), [(2 * nlayers, n)])

In [9]:
# keras interface with keras training paradigm

model1 = tf.keras.Sequential([ql])

model1.compile(
    loss="binary_crossentropy", #Because labels are in [0, 1] range. Can be changed to hinge loss if labels are transformed to [-1, 1]. Related: https://www.tensorflow.org/quantum/tutorials/mnist#2_quantum_neural_network
    optimizer=tf.keras.optimizers.legacy.Adam(0.01),
    metrics=["binary_accuracy"],
)

model1.fit(x_train, y_train, batch_size=32, epochs=20)

Epoch 1/20
125/125 [==============================] - 12s 5ms/step - loss: 0.6785 - binary_accuracy: 0.5990
Epoch 2/20
125/125 [==============================] - 1s 6ms/step - loss: 0.6529 - binary_accuracy: 0.6925
Epoch 3/20
125/125 [==============================] - 1s 6ms/step - loss: 0.6464 - binary_accuracy: 0.6908
Epoch 4/20
125/125 [==============================] - 1s 6ms/step - loss: 0.6451 - binary_accuracy: 0.7028
Epoch 5/20
125/125 [==============================] - 1s 8ms/step - loss: 0.6440 - binary_accuracy: 0.7028
Epoch 6/20
125/125 [==============================] - 1s 6ms/step - loss: 0.6443 - binary_accuracy: 0.7057
Epoch 7/20
125/125 [==============================] - 1s 6ms/step - loss: 0.6436 - binary_accuracy: 0.7067
Epoch 8/20
125/125 [==============================] - 1s 6ms/step - loss: 0.6430 - binary_accuracy: 0.7103
Epoch 9/20
125/125 [==============================] - 1s 6ms/step - loss: 0.6431 - binary_accuracy: 0.7107
Epoch 10/20
125/125 [===============

In [10]:
model1.evaluate(x_test, y_test)

62/62 [==============================] - 8s 2ms/step - loss: 0.6373 - binary_accuracy: 0.7208


[0.6373297572135925, 0.7207661271095276]

### Classical Neural Network

In [11]:
neural_network = NeuralNetwork(input_shape=(n,))

In [12]:
neural_network.train(x_train, y_train)

Epoch 1/100
125/125 [==============================] - 0s 580us/step - loss: 7.9207 - binary_accuracy: 0.4865
Epoch 2/100
125/125 [==============================] - 0s 521us/step - loss: 7.9207 - binary_accuracy: 0.4865
Epoch 3/100
125/125 [==============================] - 0s 520us/step - loss: 7.9207 - binary_accuracy: 0.4865
Epoch 4/100
125/125 [==============================] - 0s 519us/step - loss: 7.9207 - binary_accuracy: 0.4865
Epoch 5/100
125/125 [==============================] - 0s 511us/step - loss: 7.9207 - binary_accuracy: 0.4865
Epoch 6/100
125/125 [==============================] - 0s 524us/step - loss: 7.9207 - binary_accuracy: 0.4865
Epoch 7/100
125/125 [==============================] - 0s 518us/step - loss: 7.9207 - binary_accuracy: 0.4865
Epoch 8/100
125/125 [==============================] - 0s 530us/step - loss: 7.9207 - binary_accuracy: 0.4865
Epoch 9/100
125/125 [==============================] - 0s 661us/step - loss: 7.9207 - binary_accuracy: 0.4865
Epoch 10/1

Somehow not working with $3 \times 3$ images, but it works with $4 \times 4.$

In [13]:
neural_network.model.evaluate(x_test, y_test)

62/62 [==============================] - 0s 544us/step - loss: 7.9163 - binary_accuracy: 0.4868


[7.916258811950684, 0.48678863048553467]